In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(123)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools
from scipy.optimize import linear_sum_assignment
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical # used for converting labels to one-hot-encoding
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input,Dense, Dropout, Flatten, Conv2D, MaxPool2D,Conv2DTranspose,Reshape
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import itertools
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

In [2]:
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')
y_test = [np.argmax(y)for y in y_test]
y_train = [np.argmax(y)for y in y_train]
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

In [3]:
def acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Require scikit-learn installed
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`
    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_sum_assignment(w.max() - w)
    ind = np.asarray(ind)
    ind = np.transpose(ind)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

In [4]:
n_clusters = len(np.unique(y))
def DeepCluster(input_shape=(75, 100, 3), filters=[32, 64, 128, 7]):
    input_img = Input(shape=input_shape)

    x = Conv2D(filters[0],kernel_size=(3, 3), padding='same', activation='relu', name='conv1', input_shape=input_shape)(input_img)
    x = Conv2D(filters[0], kernel_size=(3, 3), padding='same', activation='relu', name='conv2')(x)
    x = MaxPool2D(pool_size = (2, 2))(x)
    
    x = Conv2D(filters[1], kernel_size=(3, 3), padding='same', activation='relu', name='conv3')(x)
    x = Conv2D(filters[1], kernel_size=(3, 3), padding='same', activation='relu', name='conv4')(x)
    x = MaxPool2D(pool_size = (2, 2))(x)
    
    x = Conv2D(filters[2], kernel_size=(3, 3), padding='same', activation='relu', name='conv5')(x)
    x = Conv2D(filters[2], kernel_size=(3, 3), padding='same', activation='relu', name='conv6')(x)
    x = MaxPool2D(pool_size = (2, 2))(x)
    
    x = Flatten()(x)
    conv_output = Dense(filters[2], activation='relu')(x)
    classifier = Dense(filters[3], activation='softmax')(conv_output)
    return  [Model(inputs=input_img, outputs=conv_output, name='conv_output'),Model(inputs=input_img, outputs=classifier, name='classifier')]

In [5]:
conv_net, Deep_cluster = DeepCluster()
conv_net.summary()
Deep_cluster.summary()

Model: "conv_output"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 75, 100, 3)]      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 75, 100, 32)       896       
_________________________________________________________________
conv2 (Conv2D)               (None, 75, 100, 32)       9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 37, 50, 32)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 37, 50, 64)        18496     
_________________________________________________________________
conv4 (Conv2D)               (None, 37, 50, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 25, 64)        

In [6]:
Deep_cluster.compile(optimizer=tf.keras.optimizers.SGD(0.001, 0.9), loss='kld')

In [7]:
def target_distribution(q):
    temp =1/(1+q**2)
    temp=temp.transpose()/temp.sum(1)
    temp=temp.transpose()
    return temp

In [8]:
print(n_clusters)
#x_1 = x.reshape((x.shape[0], -1))
#kmeans = KMeans(n_clusters=n_clusters, n_init=7)
#y_pred_last = kmeans.fit_predict(x_1)
y_pred_last = np.zeros(10015)
loss = 0
index = 0
maxiter = 8000
batch_size = 10
update_interval = 60
index_array = np.arange(x.shape[0])
tol = 0.1 # tolerance threshold to stop training

for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        temp_feature = conv_net.predict(x, verbose=0)
        y_pred = kmeans.predict(temp_feature)
        q = kmeans.transform(temp_feature)
        p = target_distribution(q)# update the auxiliary target distribution p
        # evaluate the clustering performance
        if y is not None:
            accuracy = np.round(acc(y, y_pred), 5)
            loss = np.round(loss, 5)
            print('Iter %d: accuracy = %.5f,' % (ite, accuracy), ' ; loss=%f'%(loss))

        # check stop criterion
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        print(delta_label)
        y_pred_last = np.copy(y_pred)
        if (ite > 0 and delta_label < tol):
            if(delta_label < tol):
                print('delta_label ', delta_label, '< tol ', tol)
                print('Reached tolerance threshold. Stopping training.')
            break
    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    loss = Deep_cluster.train_on_batch(x=x[idx], y=p[idx])
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

Deep_cluster.save_weights('./Deep_cluster_model_final.h5')

7


ValueError: Incorrect number of features. Got 128 features, expected 22500